# Edit File Tool with E2B Sandbox

Demonstration of the edit_file tool functionality using E2B sandbox to modify existing files by replacing specific text content.

In [1]:
// Import E2B SDK and load environment
import { Sandbox } from "https://esm.sh/e2b@latest";
import * as dotenv from "jsr:@std/dotenv";

// Import the edit_file tool
import { editFileTool, handleEditFile } from '../src/tools/edit_file.ts';
import { StreamData } from 'npm:ai';

// Load environment variables
await dotenv.load({ export: true });

const apiKey = Deno.env.get("E2B_API_KEY");
if (!apiKey) {
  throw new Error("E2B_API_KEY environment variable is required");
}

console.log("Environment loaded successfully");

Environment loaded successfully


In [2]:
// Create sandbox and clone repository
const sandbox = await Sandbox.create({
    template: "base",
    apiKey: apiKey,
    timeoutMs: 600_000 // 10 minutes
  });
  
  console.log(`Sandbox created with ID: ${sandbox.id}`);
  
  // Clone the Sudoku Solver repository
  const cloneResult = await sandbox.commands.run("git clone https://github.com/Rahimeen-Altaf/Sudoku-Solver-DSA.git /home/user/sudoku-solver");
  console.log("Repository cloned successfully");
  
  // Configure git
  await sandbox.commands.run("git config --global user.email 'test@example.com'");
  await sandbox.commands.run("git config --global user.name 'Test User'");
  
  console.log("Sandbox ready for testing");

  // Set up mock objects for testing
const mockData = new StreamData();
const mockMessages: any[] = [];
const mockAppendToolCallMessage = (msg: any) => [msg];

console.log('Tool definition:', editFileTool);

Sandbox created with ID: undefined
Repository cloned successfully
Sandbox ready for testing
Tool definition: {
  type: "function",
  function: {
    name: "edit_file",
    description: "Edit the contents of a file by replacing specific text with new text.",
    parameters: {
      type: "object",
      properties: {
        path: { type: "string", description: "The file path to edit" },
        old_text: { type: "string", description: "The existing text to replace" },
        new_text: {
          type: "string",
          description: "The new text to replace the old text with"
        }
      },
      required: [ "path", "old_text", "new_text" ]
    }
  }
}


In [3]:
// First, create a test file to edit
const testContent = `# Test Configuration File
# This is a sample configuration

DEBUG_MODE = False
MAX_ITERATIONS = 100
OUTPUT_FORMAT = "text"

# End of configuration`;

await sandbox.files.write('/home/user/sudoku-solver/test_config.py', testContent);
console.log('Created test file with initial content:');
console.log(testContent);

Created test file with initial content:
# Test Configuration File
# This is a sample configuration

DEBUG_MODE = False
MAX_ITERATIONS = 100
OUTPUT_FORMAT = "text"

# End of configuration


In [4]:
// Test basic text replacement
const basicEditToolCall = {
  id: 'test-basic-edit',
  func: {
    name: 'edit_file',
    arguments: {
      path: '/home/user/sudoku-solver/test_config.py',
      old_text: 'DEBUG_MODE = False',
      new_text: 'DEBUG_MODE = True'
    }
  }
};

console.log('=== Testing Basic Text Replacement ===');
const basicResult = await handleEditFile(
  basicEditToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Edit result:', basicResult[0].tool_call_result);

// Verify the change
const updatedContent = await sandbox.files.read('/home/user/sudoku-solver/test_config.py');
console.log('Updated file content:');
console.log(updatedContent);

=== Testing Basic Text Replacement ===
Edit result: {
  success: true,
  message: 'File edited successfully: replaced "DEBUG_MODE = False" with "DEBUG_MODE = True" in /home/user/sudoku-solver/test_config.py'
}
Updated file content:
# Test Configuration File
# This is a sample configuration

DEBUG_MODE = True
MAX_ITERATIONS = 100
OUTPUT_FORMAT = "text"

# End of configuration


In [5]:
// Test replacing multiple words
const multiWordEditToolCall = {
  id: 'test-multi-word',
  func: {
    name: 'edit_file',
    arguments: {
      path: '/home/user/sudoku-solver/test_config.py',
      old_text: 'OUTPUT_FORMAT = "text"',
      new_text: 'OUTPUT_FORMAT = "json"\nVERBOSE_LOGGING = True'
    }
  }
};

console.log('=== Testing Multi-word Replacement with New Line ===');
const multiWordResult = await handleEditFile(
  multiWordEditToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Multi-word edit result:', multiWordResult[0].tool_call_result);

// Verify the change
const updatedContent2 = await sandbox.files.read('/home/user/sudoku-solver/test_config.py');
console.log('Updated file content:');
console.log(updatedContent2);

=== Testing Multi-word Replacement with New Line ===
Multi-word edit result: {
  success: true,
  message: 'File edited successfully: replaced "OUTPUT_FORMAT = "text"" with "OUTPUT_FORMAT = "json"\n' +
    'VERBOSE_LOGGING = True" in /home/user/sudoku-solver/test_config.py'
}
Updated file content:
# Test Configuration File
# This is a sample configuration

DEBUG_MODE = True
MAX_ITERATIONS = 100
OUTPUT_FORMAT = "json"
VERBOSE_LOGGING = True

# End of configuration


In [6]:
// Test error handling - text not found
const notFoundToolCall = {
  id: 'test-not-found',
  func: {
    name: 'edit_file',
    arguments: {
      path: '/home/user/sudoku-solver/test_config.py',
      old_text: 'NON_EXISTENT_CONFIG = True',
      new_text: 'REPLACEMENT_CONFIG = False'
    }
  }
};

console.log('=== Testing Error Handling - Text Not Found ===');
const notFoundResult = await handleEditFile(
  notFoundToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('Not found error result:', notFoundResult[0].tool_call_result);

=== Testing Error Handling - Text Not Found ===
Not found error result: {
  error: 'Text "NON_EXISTENT_CONFIG = True" not found in file /home/user/sudoku-solver/test_config.py'
}


In [7]:
// Test error handling - file not found
const fileNotFoundToolCall = {
  id: 'test-file-not-found',
  func: {
    name: 'edit_file',
    arguments: {
      path: '/home/user/sudoku-solver/non_existent_file.py',
      old_text: 'some text',
      new_text: 'new text'
    }
  }
};

console.log('=== Testing Error Handling - File Not Found ===');
const fileNotFoundResult = await handleEditFile(
  fileNotFoundToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('File not found error result:', fileNotFoundResult[0].tool_call_result);

=== Testing Error Handling - File Not Found ===
File not found error result: {
  error: "path '/home/user/sudoku-solver/non_existent_file.py' does not exist"
}


In [8]:
// Test editing the actual README.md from the repository
const readmeEditToolCall = {
  id: 'test-readme-edit',
  func: {
    name: 'edit_file',
    arguments: {
      path: '/home/user/sudoku-solver/README.md',
      old_text: 'Sudoku Solver',
      new_text: 'Advanced Sudoku Solver with AI'
    }
  }
};

console.log('=== Testing Real File Edit - README.md ===');
const readmeResult = await handleEditFile(
  readmeEditToolCall,
  mockData,
  mockMessages,
  mockAppendToolCallMessage,
  sandbox
);

console.log('README edit result:', readmeResult[0].tool_call_result);

// Show the first few lines of the modified README
const modifiedReadme = await sandbox.files.read('/home/user/sudoku-solver/README.md');
console.log('Modified README preview:', modifiedReadme.substring(0, 200) + '...');

=== Testing Real File Edit - README.md ===
README edit result: {
  success: true,
  message: 'File edited successfully: replaced "Sudoku Solver" with "Advanced Sudoku Solver with AI" in /home/user/sudoku-solver/README.md'
}
Modified README preview: # Advanced Sudoku Solver with AI

The Sudoku Solver project is an advanced Python application designed to efficiently solve Sudoku puzzles. Sudoku is a number puzzle game played on a 9x9 grid, divid...


In [9]:
// Clean up sandbox
await sandbox.close();
console.log('Sandbox closed successfully');

TypeError: sandbox.close is not a function